# データの扱い方
データは機械学習モデルを構築する基盤です。データをクラウドで一元管理し、複数のワークステーションやコンピュートターゲットで実験やトレーニングモデルを実行するデータサイエンティストのチームがアクセスできるようにすることは、プロフェッショナルなデータサイエンスソリューションの重要な部分です。

このノートブックでは、データを扱うための2つのAzure Machine Learningオブジェクトについて説明します： *データストア*と*データアセット*です。

## 始める前に

このノートブックのコードを実行するには、**azure-ai-ml** パッケージの最新バージョンが必要です。以下のセルを実行して、インストールされていることを確認してください。

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、識別子のパラメータ（サブスクリプションID、リソースグループ名、ワークスペース名）が必要です。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、ワークスペースに接続するにはデフォルト値を使用できる。

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()


In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## List the datastores

Azure Machine Learning ワークスペースを作成すると、Azure Storage Account も作成される。Storage Account には Blob ストレージとファイル ストレージが含まれ、自動的に **datastores** としてワークスペースに接続される。ワークスペースに接続されているすべてのデータストアを一覧できます：

In [4]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

azureml_globaldatasets
workspaceartifactstore
workspaceworkingdirectory
workspacefilestore
workspaceblobstore


workspaceblobstore`は、先ほど説明した**azureml-blobstore-...**コンテナに接続していることに注意してください。workspacefilestore`は**code-...**ファイル共有に接続する。

## Create a datastore

別の Azure ストレージサービスと Azure Machine Learning ワークスペースを接続したい場合は、いつでもデータストアを作成できる。データストアを作成すると、ワークスペースとストレージ間の接続が作成され、ストレージサービス自体は作成されないことに注意してください。

データストアを作成し、（既に存在する）ストレージに接続するには、以下のように指定する必要がある：

- 接続したいストレージサービスのタイプを示すクラス。以下の例では Blob ストレージ (`AzureBlobDatastore`) に接続しています。
- name`： Azure Machine Learning ワークスペースのデータストアの表示名。
- description`： データストアの詳細情報を提供するためのオプションの説明。
- アカウント名`： Azure Storage Account の名前。
- コンテナ名`： Azure Storage Account で blob を保存するコンテナの名前。
- 認証情報`： 認証方法と認証するための認証情報を指定する。以下の例では、アカウントキーを使用している。

**重要**： 
- YOUR-STORAGE-ACCOUNT-NAME** は、自動的に作成されたストレージアカウント名に置き換えます。
- account_key`の **XXXX-XXXX** は、Azure ストレージアカウントのアカウントキーに置き換えます。

アカウントキーは、[Azure ポータル](https://portal.azure.com) に移動して、ストレージアカウントに移動し、**Access keys** タブから、key1 または key2 の **Key** 値をコピーすることで取得できます。


In [7]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

store = AzureBlobDatastore(
    name="blob_training_data",
    description="Blob Storage for training data",
    account_name="koikedemo9278802619",
    container_name="training-data", 
    credentials=AccountKeyConfiguration(
        account_key="keyをコピー"
    ),
)

ml_client.create_or_update(store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'blob_training_data', 'description': 'Blob Storage for training data', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/datastores/blob_training_data', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/demokoike/code/Users/wwe.ayumu/azure-ml-labs/Labs/03', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f4b46e2f6d0>, 'credentials': <azure.ai.ml.entities._credentials.NoneCredentialConfiguration object at 0x7f4b5e9fbeb0>, 'container_name': 'training-data', 'account_name': 'koikedemo9278802619', 'endpoint': 'core.windows.net', 'protocol': 'https'})

データストアをもう一度リストアップして、`blob_training_data`という名前の新しいデータストアが作成されていることを確認します：

In [8]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_training_data
azureml_globaldatasets
workspaceartifactstore
workspaceworkingdirectory
workspacefilestore
workspaceblobstore


## Create data assets

データストア内の特定のフォルダやファイルを指定するには、データアセットを作成します。データアセットには3つのタイプがあります：

- URI_FILE` は特定のファイルを指す。
- URI_FOLDER` は特定のフォルダを指す。
- MLTABLE` は MLTable ファイルを指し、フォルダ内の 1 つまたは複数のファイルの読み取り方法を指定する。

この3つのタイプのデータ資産をすべて作成し、それぞれの違いを体験してください。

URI_FILE` データアセットを作成するには、特定のファイルを指すパスを指定する必要があります。パスはローカルパスでもクラウドパスでもかまいません。

以下の例では、*ローカル*パスを参照してデータアセットを作成します。Azure Machine Learningワークスペースで作業するときにデータが常に利用できるように、ローカルファイルはデフォルトのデータストアに自動的にアップロードされます。この場合、`diabetes.csv` ファイルは、**workspaceblobstore** データストアの **LocalUpload** フォルダにアップロードされます。

ローカルファイルからデータアセットを作成するには、以下のセルを実行します：

In [9]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './data/diabetes.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Data asset pointing to a local file, automatically uploaded to the default datastore",
    name="diabetes-local"
)

ml_client.data.create_or_update(my_data)

Uploading diabetes.csv (< 1 MB): 100%|██████████| 518k/518k [00:00<00:00, 10.3MB/s]




Data({'path': 'azureml://subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo/datastores/workspaceblobstore/paths/LocalUpload/d52d15a0d7d1e95b90a03f146099424a/diabetes.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-local', 'description': 'Data asset pointing to a local file, automatically uploaded to the default datastore', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/data/diabetes-local/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/demokoike/code/Users/wwe.ayumu/azure-ml-labs/Labs/03', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f4b46e56800>, 

URI_FOLDER` データアセットを作成するには、特定のフォルダを指すパスを指定する必要があります。パスはローカルパスでもクラウドパスでもかまいません。

以下の例では、*クラウド*パスを参照してデータアセットを作成します。パスはまだ存在する必要はありません。フォルダは、データがパスにアップロードされたときに作成されます。

In [10]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

datastore_path = 'azureml://datastores/blob_training_data/paths/data-asset-path/'

my_data = Data(
    path=datastore_path,
    type=AssetTypes.URI_FOLDER,
    description="Data asset pointing to data-asset-path folder in datastore",
    name="diabetes-datastore-path"
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo/datastores/blob_training_data/paths/data-asset-path/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-datastore-path', 'description': 'Data asset pointing to data-asset-path folder in datastore', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/data/diabetes-datastore-path/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/demokoike/code/Users/wwe.ayumu/azure-ml-labs/Labs/03', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f4b46ed3c10>, 'serialize': <msrest.serialization.Serializer o

MLTable`データ資産を作成するには、MLTableファイルを含むフォルダを指すパスを指定する必要があります。パスはローカルパスでもクラウドパスでもかまいません。

> 注**：
> MLTable` ファイルを `MLTable.yaml` や `MLTable.yml` にリネームしないでください。Azure の機械学習では、`MLTable` ファイルが必要です。

以下の例では、MLTable と CSV ファイルを含む *ローカル* パスを参照してデータアセットを作成します。

In [11]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_path = 'data/'

my_data = Data(
    path=local_path,
    type=AssetTypes.MLTABLE,
    description="MLTable pointing to diabetes.csv in data folder",
    name="diabetes-table"
)

ml_client.data.create_or_update(my_data)

Uploading data (0.52 MBs): 100%|██████████| 518211/518211 [00:00<00:00, 19717817.60it/s]




Data({'path': 'azureml://subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo/datastores/workspaceblobstore/paths/LocalUpload/45c4bb11b5759d37974b72b7b978482e/data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['./diabetes.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-table', 'description': 'MLTable pointing to diabetes.csv in data folder', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/data/diabetes-table/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/demokoike/code/Users/wwe.ayumu/azure-ml-labs/Labs/03', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f4b46ed4670>, 'serialize': <msrest.serializat

To verify that the new data assets have been created, you can list all data assets in the workspace again:

In [12]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

diabetes
diabetes-local
diabetes-datastore-path
diabetes-table


## Read data in notebook

## ノートブックでデータを読む

最初はノートブックでデータ資産を操作して、データを探索したり機械学習モデルを実験したりしたいと思うかもしれません。URI_FILE` または `URI_FOLDER` タイプのデータアセットは、通常のデータ読み込みと同様に読み込まれます。例えば、データ資産が指すCSVファイルを読み込むには、pandasの関数 `read_csv()` を使用します。

MLTable` 型のデータアセットは、スキーマとデータの解釈方法を指定する **MLTable** ファイルによって既に **読み込まれ* ています。このファイルはスキーマを指定し、データをどのように解釈するかを指定します。データはすでに *read* されているので、MLTable 型のデータ資産を簡単に pandas dataframe に変換することができます。

mltable`ライブラリをインストールする必要があります（ターミナルでインストールしました）。その後、データ資産をデータフレームに変換し、データを可視化することができます。

In [14]:
pip install mltable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
import mltable

registered_data_asset = ml_client.data.get(name='diabetes-table', version=1)
tbl = mltable.load(f"azureml:/{registered_data_asset.id}")
df = tbl.to_pandas_dataframe()
df.head(5)

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,False
1,1147438,8,92,93,47,36,21.240576,0.158365,23,False
2,1640031,7,115,47,52,35,41.511523,0.079019,23,False
3,1883350,9,103,78,25,304,29.582192,1.282870,43,True
4,1424119,1,85,59,27,35,42.604536,0.549542,22,False


## Use data in a job

ノートブックを実験に使った後 スクリプトを使って機械学習モデルを学習することができる。スクリプトはジョブとして実行でき、ジョブごとに入力と出力を指定できます。

ジョブの入力または出力として、**データアセット**または**データストアパス**のいずれかを使用できます。

以下のセルは **src** フォルダに **move-data.py** スクリプトを作成します。スクリプトは `read_csv()` 関数で入力データを読み込みます。スクリプトはデータを CSV ファイルとして出力パスに格納します。

In [16]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [17]:
%%writefile $script_folder/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

def main(args):
    # read data
    df = get_data(args.input_data)

    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Writing src/move-data.py


move-data.py**スクリプトを実行するジョブを投入するには、以下のセルを実行してください。

このジョブはデータアセット `diabetes-local` を使用するように設定されており、入力としてローカルの **diabetes.csv** ファイルを指します。出力は新しいデータストア `blob_training_data` のフォルダを指すパスです。

In [22]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command

# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-local:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/blob_training_data/paths/datastore-path")
}

# configure job
#Computename を自身のコンピューティングインスタンス名に置き換えてください。
job = command(
    code="./src",
    command="python move-data.py --input_data ${{inputs.local_data}} --output_datastore ${{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="demokoike",
    display_name="move-diabetes-data",
    experiment_name="move-diabetes-data"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Monitor your job at https://ml.azure.com/runs/orange_night_lh9tdjlryq?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1
